<a href="https://colab.research.google.com/github/DianaMoyano1/NLP-Sentiment_Extraction_Challenge/blob/master/Tutorial_SingleM_Template_Eman_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP


1. Click on Runtime --> Change runtime type
2. Select **GPU** under *Hardware accelerator*



Before running this notebook, make sure you have added the *'tweet-sentiment-extraction-tutorial'* shortcut to your Gdrive, inside the *'Colab Notebooks'* directory

![](https://serving.photos.photobox.com/817973499a3406a94b0556385350836dda810c16c1a20967bd25b50dd6e367b3dcb1e30b.jpg)

### Mount Your Own Gdrive

Below command will require you to validate your account, and it will provide you with a temporary access code to paste in the field

In [ ]:
# Mount your local Google drive and show the models you have
from google.colab import drive
drive.mount('/gdrive')
%ls '/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction-tutorial/models' 

In [ ]:
#install the following packages. The --quiet command will reduce the output lines
!pip install transformers==2.11.0 --quiet
!pip install tensorflow==2.2.0 --quiet
!pip install tensorboardX --quiet
!pip install simpletransformers --quiet
!pip install chart_studio --quiet

### Setup NVIDIA APEX

Tool to enable mixed precision training in Pytorch (the underlying structure for SimpleTransformers). More info here: https://github.com/NVIDIA/apex

In [ ]:
%%writefile setup.sh
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
#this will take 7-10 mins to run
import timeit
start = timeit.default_timer()

!sh setup.sh --quiet

stop = timeit.default_timer()
print('Time: ', stop - start)  

### Import Packages

In [ ]:
#Import packages
from os.path import join
import numpy as np 
import pandas as pd 
from apex import amp
import json


use_cuda = True ##If True, GPU will be used

### Load the Data

In [ ]:
train_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction-tutorial/train.csv')
test_df = pd.read_csv('/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction-tutorial/test.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
#@title Import Packages
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from google.colab import files
import pandas as pd
import io

# Visualisation libraries
import matplotlib.pyplot as plt 
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# sklearn 
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

#Text Pre-processing
import nltk
import re

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Make a copy of the original datasets for visualization purposes
train = train_df.copy()
test = test_df.copy()

In [ ]:
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [ ]:
#@title Sentiment Distribution


# Sentiment distribution in training set
enable_plotly_in_cell()

train['sentiment'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='red',
                                                      theme='pearl',
                                                      bargap=0.6,
                                                      gridcolor='white',
                                                      dimensions=(1000,500),
                                                     
                                                      title='Distribution of Sentiment column in the training set')


# Sentiment distribution in test set
enable_plotly_in_cell()

test['sentiment'].value_counts(normalize=True).iplot(kind='bar', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.6,
                                                      gridcolor='white',
                                                      dimensions=(1000,500),
                                                      title='Distribution  of Sentiment column in the test set')

In [ ]:
#@title Text Pre-processing Functions


def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

### Training Dataset

In [ ]:
#@title Applying the cleaning function to the training dataset
train['text_clean'] = train['text'].apply(str).apply(lambda x: text_preprocessing(x))
test['text_clean'] = test['text'].apply(str).apply(lambda x: text_preprocessing(x))

train['text_len'] = train['text_clean'].astype(str).apply(len)
train['text_word_count'] = train['text_clean'].apply(lambda x: len(str(x).split()))

pos = train[train['sentiment']=='positive']
neg = train[train['sentiment']=='negative']
neutral = train[train['sentiment']=='neutral']

In [ ]:
#@title Box Plots
# Text length box plot

enable_plotly_in_cell()

trace0 = go.Box(
    y=pos['text_len'],
    name = 'Positive Text',
    marker = dict(
        color = 'blue',
    )
)

trace1 = go.Box(
    y=neg['text_len'],
    name = 'Negative Text',
    marker = dict(
        color = 'red',
    )
)

trace2 = go.Box(
    y=neutral['text_len'],
    name = 'Neutral Text',
    marker = dict(
        color = 'orange',
    )
)
data = [trace0, trace1, trace2]
layout = go.Layout(
    title = "Length of the text"
)

fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = "Length of the text of different polarities")


#Word count box plot

enable_plotly_in_cell()

trace0 = go.Box(
    y=pos['text_word_count'],
    name = 'Positive Text',
    marker = dict(
        color = 'blue',
    )
)

trace1 = go.Box(
    y=neg['text_word_count'],
    name = 'Negative Text',
    marker = dict(
        color = 'red',
    )
)

trace2 = go.Box(
    y=neutral['text_word_count'],
    name = 'Neutral Text',
    marker = dict(
        color = 'orange',
    )
)
data = [trace0, trace1, trace2]
layout = go.Layout(
    title = "word count of the text"
)

fig = go.Figure(data=data,layout=layout)
iplot(fig, filename = "word count of the text of different polarities")

In [ ]:
#@title Function: Get Top K-Words
#source of code : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_k_words(corpus, n=None):

    vec = CountVectorizer(stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#@title Top 20 Words per Sentiment
enable_plotly_in_cell()

pos_unigrams = get_top_k_words(pos['text_clean'],20)
neg_unigrams = get_top_k_words(neg['text_clean'],20)
neutral_unigrams = get_top_k_words(neutral['text_clean'],20)


df1 = pd.DataFrame(pos_unigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='blue', title='Positive Sentiment',orientation='h')

df2 = pd.DataFrame(neg_unigrams, columns = ['Text' , 'count'])
df2.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='red',title='Negative Sentiment',orientation='h')

df3 = pd.DataFrame(neutral_unigrams, columns = ['Text' , 'count'])
df3.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Neutral Sentiment',orientation='h')

In [ ]:
#@title Function: Top N-Grams Function 
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
#@title Top 20 Bi-grams
enable_plotly_in_cell()

pos_bigrams = get_top_n_gram(pos['text_clean'],(2,2),20)
neg_bigrams = get_top_n_gram(neg['text_clean'],(2,2),20)
neutral_bigrams = get_top_n_gram(neutral['text_clean'],(2,2),20)



#for word, freq in top_bigrams:
    #print(word, freq)
df1 = pd.DataFrame(pos_bigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='blue', title='Top 20 Bigrams in positve text',orientation='h')

df2 = pd.DataFrame(neg_bigrams, columns = ['Text' , 'count'])
df2.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='red',title='Top 20 Bigrams in negative text',orientation='h')

df3 = pd.DataFrame(neutral_bigrams, columns = ['Text' , 'count'])
df3.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 Bigrams in neutral text',orientation='h')

In [ ]:
#@title Top 20 Tri-grams

enable_plotly_in_cell()

pos_trigrams = get_top_n_gram(pos['text_clean'],(3,3),20)
neg_trigrams = get_top_n_gram(neg['text_clean'],(3,3),20)
neutral_trigrams = get_top_n_gram(neutral['text_clean'],(3,3),20)

df1 = pd.DataFrame(pos_trigrams, columns = ['Text' , 'count'])
df1.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black',color='blue', title='Top 20 Trigrams in positve text',orientation='h')

df2 = pd.DataFrame(neg_trigrams, columns = ['Text' , 'count'])
df2.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', color='red',title='Top 20 Trigrams in negative text',orientation='h')

df3 = pd.DataFrame(neutral_trigrams, columns = ['Text' , 'count'])
df3.groupby('Text').sum()['count'].sort_values(ascending=True).iplot(
    kind='bar', yTitle='Count', linecolor='black', title='Top 20 Trigrams in neutral text',orientation='h')

### Word Clouds

In [ ]:
#@title Word Cloud Data Prep
positive_text = train[train['sentiment'] == 'positive']['selected_text']
negative_text = train[train['sentiment'] == 'negative']['selected_text']
neutral_text = train[train['sentiment'] == 'neutral']['selected_text']

positive_text_clean = positive_text.apply(lambda x: text_preprocessing(x))
negative_text_clean = negative_text.apply(lambda x: text_preprocessing(x))
neutral_text_clean = neutral_text.apply(lambda x: text_preprocessing(x))


#Apply the top_n_words function

top_words_in_positive_text = get_top_k_words(positive_text_clean)
top_words_in_negative_text = get_top_k_words(negative_text_clean)
top_words_in_neutral_text = get_top_k_words(neutral_text_clean)

p1 = [x[0] for x in top_words_in_positive_text[:20]]
p2 = [x[1] for x in top_words_in_positive_text[:20]]


n1 = [x[0] for x in top_words_in_negative_text[:20]]
n2 = [x[1] for x in top_words_in_negative_text[:20]]


nu1 = [x[0] for x in top_words_in_neutral_text[:20]]
nu2 = [x[1] for x in top_words_in_neutral_text[:20]]


In [ ]:
#@title Plot Word Clouds
from wordcloud import WordCloud
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=[30, 15])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(positive_text_clean))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('Positive text',fontsize=40);

wordcloud2 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(negative_text_clean))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Negative text',fontsize=40);

wordcloud3 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(neutral_text_clean))
ax3.imshow(wordcloud3)
ax3.axis('off')
ax3.set_title('Neutral text',fontsize=40);

# DATA PREP

Split into train and validation sets

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state = 42)

In [ ]:
#drop selected_text column from the validation dataset (it will be added back once we are comparing it to our predictions)
val_df_new = val_df.drop('selected_text', axis=1)

In [ ]:
print(train_df.shape)
print(val_df_new.shape)
print(test_df.shape)

In [ ]:
train = np.array(train_df)
val = np.array(val_df_new)
test = np.array(test_df)

### Initiate the SimpleTransformers Task



The SimpleTransformers library supports numerous tasks:  


- Sequence Classification
- Token Classification (NER)
- Question Answering
- Language Model Fine-Tuning
- Language Model Training
- Language Generation
- T5 Model
- Seq2Seq Tasks
- Multi-Modal Classification
- Conversational AI

In this case, we are performing a <ins>Question Answering</ins> task.

Supported model types:

- ALBERT
- BERT
- DistilBERT
- ELECTRA
- XLM
- XLNet

In [ ]:
# Import the Question Answering model
from simpletransformers.question_answering import QuestionAnsweringModel

### Format the data under the SimpleTransformer's *Question&Answer* schema 



To input the dataset, we need to assign each column to specific inputs
- Context: The entire tweet
- Question: The sentiment (positive, negative or neutral). In other words, we are asking *\"What part of the entire tweet best represents this sentiment?\"*
- Answer: the label - the extracted text

The formated data is assigned to the variables *qa_train, qa_val* and *qa_test* respectively



In [ ]:
#@title Create list for training

## Adapted from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        if type(answer) != str or type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output

qa_train = do_qa_train(train)


In [ ]:
qa_train[1:3]


In [ ]:
#@title Create val list
## Adapted from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing
def do_qa_val(val):
    output = []
    for line in val:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_val = do_qa_val(val)

In [ ]:
#@title Create test list
## Adapted from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing
def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        if type(context) != str or type(question) != str:
            print(context, type(context))
            print(answer, type(answer))
            print(question, type(question))
            continue
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

### Create a Logging Module --> More info [here](https://realpython.com/python-logging/#:~:text=The%20Logging%20Module,-The%20logging%20module&text=It%20is%20used%20by%20most,homogeneous%20log%20for%20your%20application.&text=With%20the%20logging%20module%20imported,that%20you%20want%20to%20see.)


Logs provide developers with an extra set of eyes that are constantly looking at the flow that an application is going through. They can store information, like which user or IP accessed the application.  

With the logging module imported, you can use something called a “logger” to log messages that you want to see. By default, there are 5 standard levels indicating the severity of events.
- DEBUG
- INFO
- WARNING
- ERROR
- CRITICAL

In this case, we picked INFO and WARNING

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

### Follow the section that best applies to your case
Load and Evaluate a Richardson's Pre-Trained Model (OPTION 1) 

**<ins>OR</ins>**

Load, Train and Evaluate a SimpleTransformers' Model (OPTION 2)

# OPTION 1: Load and Evaluate a Richardson's Pre-Trained Model

This model is available in the shared folder, under *models*

In [ ]:

ROOT= '/gdrive/My Drive/Colab Notebooks/tweet-sentiment-extraction-tutorial/models'
FOLDER_NAME= 'richardson_distilbert-base-uncased-distilled-squad_A'

FULL_PATH = join(ROOT, FOLDER_NAME)

#Change the workspace to the model folder
%cd '{FULL_PATH}' 

#Load the model's arguments list (required to setup the existing model) 
with open('args_train.json') as json_file: 
    train_args = json.load(json_file) 

## Setup and load model

Supported model types for Question&Answering:

- ALBERT
- BERT
- DistilBERT
- ELECTRA
- XLM
- XLNet

#### Details of the model we will load

Model Architecture: [Distilbert](https://arxiv.org/abs/1910.01108), A faster and powerful version of BERT

Shortcut Name: distilbert-base-uncased-distilled-squad

Trained on: [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/), Standford Question Answering Dataset



Additional architectures and shortcut names [here](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# Enter model architecture name
MODEL_ARCHITECTURE='distilbert'

In [ ]:
#Load the model
loaded_model = QuestionAnsweringModel(MODEL_ARCHITECTURE, 'outputs/', args=train_args, use_cuda=use_cuda)

In [ ]:
#Predict the evaluation and test sets
predictions_val = loaded_model.predict(qa_val)
predictions_test = loaded_model.predict(qa_test)

Let's check the structure of the predictions

In [ ]:
#It displays truncated long texts
pd.set_option('display.max_colwidth',100)

#Each ID contains multiple predicted extractions and their corresponding probabilities (prediction with highest probability is first)
pd.DataFrame.from_dict(predictions_val)[1]

Below commands will select the extracted text with the highest likelyhood (first item), as well as its corresponding probability

In [ ]:
#@title Obtain output with the highest prob - Validation set
predictions_df_val = pd.DataFrame.from_dict(predictions_val)
text_val = pd.DataFrame(predictions_val[0])
prob_val = pd.DataFrame(predictions_val[1])
prop1_val = prob_val['probability'].tolist()
prop2_val = pd.DataFrame(prop1_val)
text1_val = text_val['answer'].tolist()
text2_val = pd.DataFrame(text1_val)


In [ ]:
#@title Obtain output with the highest prob - Test set
predictions_df_test = pd.DataFrame.from_dict(predictions_test)
text_test = pd.DataFrame(predictions_test[0])
prob_test = pd.DataFrame(predictions_test[1])
prop1_test = prob_test['probability'].tolist()
prop2_test = pd.DataFrame(prop1_test)
text1_test = text_test['answer'].tolist()
text2_test = pd.DataFrame(text1_test)

In [ ]:
# Make a copy of the validation and test sets so that we are not modifying the original sets
sub_val_df = val_df.copy()
sub_test_df = test_df.copy()

In [ ]:
#Add the predicted result to the copied data frames 
sub_val_df['predicted_selected_text'] = text2_val[0].values
sub_test_df['predicted_selected_text'] = text2_test[0].values

In [ ]:
#Add the probability of the prediction
sub_val_df['prob'] = prop2_val[0].values
sub_test_df['prob'] = prop2_test[0].values

## Evaluate Validation Test with Jaccard Score

In [ ]:
# Check head of dataset
sub_val_df.head()

In [ ]:
#Make a copy of the original validation set and reset indexes
df_js=sub_val_df.copy()
df_js=df_js.reset_index()

In [ ]:
#Define the Jaccard Score function
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
#Obtain JS for the entire set
results = []
for i in range(len(df_js)):
    score = jaccard(df_js['selected_text'].iloc[i], df_js['predicted_selected_text'].iloc[i])
    results.append(score)
    
Jaccard_score = sum(results) / len(results)
Jaccard_score

## Prepare and Submit Test Set

In [ ]:
# Check head of dataset
sub_test_df.head()

In [ ]:
#Prepare file for submission
final_test=sub_test_df[['textID','predicted_selected_text']]
final_test.columns=['textID','selected_text']
final_test.head()

# OPTION 2: Load, train and evaluate a SimpleTransformers' Model

Create a folder that will contain the new model's PyTorch and hyperameters files. Follow below instructions to assign a name to the *'NAME_OF_MODEL'*  folder:


>>**Basic Structure:**

>>Name_Model_Version  

>>>Where:
- Name: Your name
- Model: Based on the model names used in the official Transformers site: https://huggingface.co/transformers/pretrained_models.html
- Version: For notebooks with same name and model but different hyperparameters, include the version (A, B, C...)
  
  >>>Examples:
  - Lucas_distilroberta-base_A
  - Lucas_distilroberta-base_B
  - Landis_bert_A  


Supported model types for Question&Answering:

- ALBERT
- BERT
- DistilBERT
- ELECTRA
- XLM
- XLNet

Under Colab Notebook, below command will create a *my-tweet-folder* directory and then a *model* folder inside of it (this is where all your models will be stored)

**Skip it if you have done this already**

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks'
%mkdir 'my-tweet-folder' 
%cd '/gdrive/My Drive/Colab Notebooks/my-tweet-folder'
%mkdir 'models'

In [ ]:
# Change this BEFORE RUNNING *********************************************************************************************
YOUR_NAME = 'richardson'
YOUR_LETTER = 'A'     # identify your model A,B,C,D,E...
MODEL_ARCHITECTURE = 'distilbert'
MODEL_NAME = 'distilbert-base-uncased-distilled-squad'
# ************************************************************************************************************************

#Don't change below lines:
FULL_NAME = YOUR_NAME + '_' + MODEL_NAME + '_' + YOUR_LETTER 


ROOT = '/gdrive/My Drive/Colab Notebooks/my-tweet-folder/models' 
FULL_PATH = join(ROOT, FULL_NAME)

Below command will create a folder where all the model's files will be stored

In [ ]:
#Change directory to "my-tweet-folder/models"
%cd '{ROOT}'
#It creates the folder where the model components will be saved. If you have a folder with the same name, it will give you an error
%mkdir '{FULL_NAME}' 
#Change the workspace to the recently created folder
%cd '{FULL_PATH}' 

In [ ]:
#For more arguments, refer to this link --> https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model

args_train={'reprocess_input_data': True,
'overwrite_output_dir': True,
'learning_rate': 5e-5,
'num_train_epochs': 1,
'max_seq_length': 192,
'doc_stride': 64,
'fp16': False,
}

#Fit the model
model = QuestionAnsweringModel(MODEL_ARCHITECTURE, MODEL_NAME, args=args_train, use_cuda=use_cuda)

In [ ]:
#This line creates a JSON file that is required to load the model in the future
with open('args_train.json', 'w') as fp: 
    json.dump(args_train, fp)

In [ ]:
#Train the model
import timeit
start = timeit.default_timer()

model.train_model(qa_train)

stop = timeit.default_timer()
print('Time: ', stop - start)  

In [ ]:
#Predict the evaluation and test sets
predictions_val = model.predict(qa_val)
predictions_test = model.predict(qa_test)


Let's check the structure of the predictions

In [ ]:
#It displays truncated long texts
pd.set_option('display.max_colwidth',100)

#Each ID contains multiple predicted extractions and their corresponding probabilities (prediction with highest probability is first)
pd.DataFrame.from_dict(predictions_val)[1]

Below commands will select the extracted text with the highest likelyhood (first item), as well as its corresponding probability

In [ ]:
#@title Obtain output with the highest prob - Validation set

#Validation Set highest probability output
predictions_df_val = pd.DataFrame.from_dict(predictions_val)
text_val = pd.DataFrame(predictions_val[0])
prob_val = pd.DataFrame(predictions_val[1])
prop1_val = prob_val['probability'].tolist()
prop2_val = pd.DataFrame(prop1_val)
text1_val = text_val['answer'].tolist()
text2_val = pd.DataFrame(text1_val)

In [ ]:
#@title Obtain output with the highest prob - Test set
predictions_df_test = pd.DataFrame.from_dict(predictions_test)
text_test = pd.DataFrame(predictions_test[0])
prob_test = pd.DataFrame(predictions_test[1])
prop1_test = prob_test['probability'].tolist()
prop2_test = pd.DataFrame(prop1_test)
text1_test = text_test['answer'].tolist()
text2_test = pd.DataFrame(text1_test)

In [ ]:
# Make a copy of the validation and test sets so that we are not modifying the original sets
sub_val_df = val_df.copy()
sub_test_df = test_df.copy()

In [ ]:
#Add the predicted result to the copied data frames 
sub_val_df['predicted_selected_text'] = text2_val[0].values
sub_test_df['predicted_selected_text'] = text2_test[0].values

In [ ]:
#Add the probability of the prediction
sub_val_df['prob'] = prop2_val[0].values
sub_test_df['prob'] = prop2_test[0].values

## Evaluate Validation Test with Jaccard Score

In [ ]:
# Check head of dataset
sub_val_df.head()

In [ ]:
#Make a copy of the original validation set and reset indexes
df_js=sub_val_df.copy()
df_js=df_js.reset_index()

In [ ]:
#Define the Jaccard Score function
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
#Obtain JS for the entire set
results = []
for i in range(len(df_js)):
    score = jaccard(df_js['selected_text'].iloc[i], df_js['predicted_selected_text'].iloc[i])
    results.append(score)
    
Jaccard_score = sum(results) / len(results)
Jaccard_score

## Prepare and Submit Test Set

In [ ]:
# Check head of dataset
sub_test_df.head()

In [ ]:
#Prepare file for submission
final_test=sub_test_df[['textID','predicted_selected_text']]
final_test.columns=['textID','selected_text']
final_test.head()

In [ ]:
#Submit
final_test[['textID','selected_text']].to_csv('submission.csv', index=False)
print("Submission successful")

## Save trained model arguments and other files

In [ ]:
#Additonal files if required
"""from google.colab import files
sub_val_df.to_csv('sub_val.csv') 
files.download('sub_val.csv')
sub_test_df.to_csv('sub_test.csv') 
files.download('sub_test.csv')
train_df.to_csv("new_train_df")"""